**Instalación de dependencias**

Agrega los paquetes necesarios a tu proyecto:

In [ ]:
%pip install azure-ai-inference[opentelemetry]
%pip install azure-search-documents 
%pip install azure-identity
%pip install openai
%pip install scikit-learn
%pip install plotly


**Configuración de la aplicación**

Aquí está la configuración de tu aplicación, podrías mejorarla usando un archivo .env en lugar de dejar aquí todas tus variables!
¿Puedes con eso?

In [21]:
import os
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import dotenv
from azure.search.documents.models import VectorizableTextQuery
import time
import numpy as np
import pandas as pd
from openai import AzureOpenAI
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go

In [22]:

dotenv.load_dotenv()

# Leer las variables definidas en el archivo .env

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME= os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")

AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")

openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
)

In [33]:
def search_with_vector_query(user_question: str, top: int = 3, k_nearest_neighbors: int = 3, vector_field: str = "text_vector"):
    """
    Realiza una búsqueda vectorial en Azure Cognitive Search usando una pregunta del usuario.

    Args:
        search_client: instancia de Azure SearchClient ya autenticada.
        user_question (str): pregunta o texto de búsqueda.
        top (int): número de documentos a recuperar.
        k_nearest_neighbors (int): número de vecinos más cercanos a buscar en el vector.
        vector_field (str): nombre del campo de vector en el índice (por defecto 'text_vector').

    Returns:
        list: lista de resultados relevantes.
    """
    search_results = search_client.search(
        None,
        top=top,
        vector_queries=[
            VectorizableTextQuery(
                text=user_question,
                k_nearest_neighbors=k_nearest_neighbors,
                fields=vector_field
            )
        ],
    )

    # Convertimos los resultados en una lista más manejable
    #for result in search_results:
    #    print("Chunk ID:", result["chunk_id"])
    #    print("Title:", result["title"])
    #    print("Text:", result["chunk"])
    #    print()

    context = ""
    for result in search_results:
        context += result["chunk"] + "\n\n"

    return context


# Comparación de Modelos LLM en RAG (Azure Search + OpenAI)

Esta sección permite comparar **tres modelos de lenguaje (LLM)** usando la misma consulta RAG y medir su rendimiento en términos de:
- Precisión semántica (similitud con la query)
- Longitud de respuesta
- Tiempo de inferencia

También puedes cambiar fácilmente el **modelo de embeddings** (por defecto `text-embedding-ada-002`).


In [14]:
# Configuración por defecto
CHAT_MODELS = ["gpt-4o", "DeepSeek-R1"]

EMBEDDINGS = ["text-embedding-ada-002"]

In [ ]:

def get_embedding(text, model=EMBEDDINGS[0]):
    response = openai_client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

def cosine(v1, v2):
    return float(cosine_similarity([v1], [v2])[0][0])



In [ ]:

def compare_models(user_question):
    results = []
    query_emb = get_embedding(user_question)

    for model_name in CHAT_MODELS:
        start = time.time()

        # Consulta los documentos relevantes usando la pregunta del usuario
        context = search_with_vector_query(user_question)

        SYSTEM_MESSAGE = f"""
        You are an AI Assistant.
        Be brief in your answers. Answer ONLY with the facts listed in the retrieved text.

        Context:
        {context}
        """

        USER_MESSAGE = user_question

        response = openai_client.chat.completions.create(
            model=model_name,
            messages=[{"role": "system", "content": SYSTEM_MESSAGE},
                      {"role": "user", "content": USER_MESSAGE}],
            temperature=0.7
        )
        
        end = time.time()

        text = response.choices[0].message.content.strip()
        resp_emb = get_embedding(text)
        sim = cosine(query_emb, resp_emb)

        results.append({
            "model": model_name,
            "response": text,
            "similarity": sim,
            "length": len(text),
            "time_sec": round(end - start, 2)
        })

    df = pd.DataFrame(results)
    print("\nResultados de comparación:")
    display(df)

    fig = go.Figure(data=[
        go.Bar(name="Similitud", x=df["model"], y=df["similarity"]),
        go.Bar(name="Longitud", x=df["model"], y=df["length"]),
        go.Bar(name="Tiempo (s)", x=df["model"], y=df["time_sec"])
    ])
    fig.update_layout(barmode="group", title="Comparativa de LLMs", xaxis_title="Modelo", yaxis_title="Valor")
    fig.show()

    return df


In [38]:

user_question = "¿En qué condiciones climáticas se recomienda sembrar aguacate?"

df_result = compare_models(user_question)


Retrieved Context for model gpt-4o :
 y rala dejando caer la 
semilla en el fondo de un pequeño 
surco a 1 o 5 centímetros de 
profundidad. Posteriormente se ralea dejando las plantas a la distancia 



Manual para el Productor: El Cultivo de las Hortalizas

Proyecto: Manejo Integral de los Recursos Naturales en el Trópico de Cochabamba y Los Yungas de La Paz

16

adecuada. Ejemplo: rábano, 
zanahoria, espinaca. En este 
sistema los cultivos de ciclo tardío 
se siembran en surcos espaciados 
a un metre entre ello y de 45 a 70 
centímetros para cultivos de ciclo 
precoz.

c) A golpes.- Se siembra la semilla en 
pequeños huecos distanciados, 
colocando 2 a 3 semillas en cada 
hueco.

4.1.2 DIMENSIONADO

Se realiza bajo el método de tres bolillos y cuadrado. En este sistema 
se hacen agujeros de 2 a 5 centímetros de profundidad. Las semillas 
o plántulas se siembran en cada punta de un triángulo o cuadrado 
imaginario. Estos sistemas permiten que más plantas puedan crecer 
dentro de un ár

,model,response,similarity,length,time_sec
0,gpt-4o,"The almacigo, or germinadero, is a small area ...",0.803226,431,3.87
1,DeepSeek-R1,The steps for indirect sowing (trasplante) are...,0.789180,407,5.41
